In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import os
import math
import mediapipe as mp

import csv


In [2]:
# MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [3]:
def extract_coordinates(video_path, video_url, class_name,start_frame, end_frame):

    if not os.path.exists("data/{}".format(class_name)):
        os.makedirs("data/{}".format(class_name))
        
    
    #creating empty file in folder, I added the start_time in the name of the csv file, so that if a symbol appears many times in a video, it will still be created in two different csv files, just that they will have different starting times
    csv_file = f"data/{class_name}/{video_url}{start_frame}.csv"
    if os.path.exists(csv_file):
        return 


# Setup CSV File for the videos
# 21 right hand landmarks, 21 left hand landmarks, 33 pose landmarks
    num_coords = 21 + 21 + 33
    landmarks = ['class']
    for val in range(1, num_coords+1):
        landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
    print("Initialized an empty landmarks of size:", len(landmarks))

    with open(csv_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)
        


#working with each video
    cap = cv2.VideoCapture(os.path.join(video_path, video_url)+".mp4")
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    # Read until video is completed
    else: 
        print("Currently working with video: ",  video_url, " ", class_name)
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

            frameNr=0

            while(cap.isOpened()):
                    
                # Capture frame-by-frame
                ret, frame = cap.read() 
                if ret == True:

                    ##!!! start countin the number of frames AFTER ret== True (which means that the frames WERE captured, therefore, you can increment)
                    frameNr+=1

                    if frameNr<start_frame:
                        print(frameNr)
                        continue

                    if frameNr>end_frame:
                        print(frameNr)
                        break

                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image.flags.writeable = False    
                    results = holistic.process(image)
                    # Display the resulting frame
                    # Right hand
                    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                            )

                    # Left Hand
                    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                            )

                    # Pose Detections
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                            )
                    cv2.imshow('Frame',image)
                    # Press Q on keyboard to  exit
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        break
                    # Export coordinates
                    try:
                        # Extract Pose landmarks
                        if results.pose_landmarks:
                            pose = results.pose_landmarks.landmark
                            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                        else:
                            # continue
                            pose_row=list(np.array([[0,0,0,0] for i in range(33)]).flatten())
                        # Extract hands landmarks
                        if results.right_hand_landmarks:
                            right_hand = results.right_hand_landmarks.landmark
                            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
                        else:
                            #If no right hand detected, then it writes 0 to the CSV file
                            right_hand_row = list(np.array([[0,0,0,0] for i in range(21)]).flatten())
                        if results.left_hand_landmarks:
                            left_hand = results.left_hand_landmarks.landmark
                            left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
                        else:
                            #If no left hand detected, then it writes 0 to the CSV file
                            left_hand_row = list(np.array([[0,0,0,0] for i in range(21)]).flatten())

                        # Concate rows
                        row = pose_row + right_hand_row + left_hand_row
                        # Append class name 
                        row.insert(0, class_name)

                        # Export to CSV
                        with open(csv_file, mode='a', newline='') as f:
                            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            csv_writer.writerow(row) 

                    except Exception as e:
                        print(str(e) +"for ", class_name, " ", video_url )
                        continue
                         
                    

            
                else:
                    break

            # When everything done, release the video capture object
            cap.release()
            # Closes all the frames
            cv2.destroyAllWindows()
            return results
